# 3B-jkk-interpret-logistic-regression

In this notebook, we will explore global and local explanations for an [interpretable model](https://christophm.github.io/interpretable-ml-book/simple.html) (Logistic Regression). We will also delve into techniques for presenting explanations to end users. Note that these local global and local explanations are unique to linear models, and other models will require other strategies.

Note that the documentation linked above may refer to a newer version of scikit-learn.

In [1]:
%config IPCompleter.use_jedi = False

from IPython.display import HTML
from joblib import load
import sqlite3 as sql
import pandas as pd
import numpy as np
import re

seed = 101

Load the entire dataset.

In [2]:
with sql.connect('../data/toxic.db') as conn:
    df = pd.read_sql_query('''select * from toxic''', conn)
df.head()

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y
0,2232.0,This:\n:One can make an analogy in mathematica...,2002,1,article,random,train,10.0,-1.0,1.0,0.4,0
1,4216.0,"""\n\n:Clarification for you (and Zundark's ri...",2002,1,user,random,train,10.0,0.0,2.0,0.5,0
2,8953.0,Elected or Electoral? JHK,2002,0,article,random,test,10.0,0.0,1.0,0.1,0
3,26547.0,"""This is such a fun entry. Devotchka\n\nI on...",2002,1,article,random,train,10.0,0.0,2.0,0.6,0
4,28959.0,Please relate the ozone hole to increases in c...,2002,1,article,random,test,10.0,-1.0,1.0,0.2,0


Split into two seperate dataframes: df_train and df_test.

In [3]:
df_train = df[df['split'] == 'train'].copy().reset_index(drop=True)
df_train.head()

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y
0,2232.0,This:\n:One can make an analogy in mathematica...,2002,1,article,random,train,10.0,-1.0,1.0,0.4,0
1,4216.0,"""\n\n:Clarification for you (and Zundark's ri...",2002,1,user,random,train,10.0,0.0,2.0,0.5,0
2,26547.0,"""This is such a fun entry. Devotchka\n\nI on...",2002,1,article,random,train,10.0,0.0,2.0,0.6,0
3,37330.0,"""\n\n\nI fixed the link; I also removed """"home...",2002,1,article,random,train,10.0,-1.0,1.0,0.1,0
4,37346.0,"""If they are """"indisputable"""" then why does th...",2002,1,article,random,train,10.0,-1.0,1.0,0.2,0


In [4]:
df_test = df[df['split'] == 'test'].copy().reset_index(drop=True)
df_test.head()

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y
0,8953.0,Elected or Electoral? JHK,2002,0,article,random,test,10.0,0.0,1.0,0.1,0
1,28959.0,Please relate the ozone hole to increases in c...,2002,1,article,random,test,10.0,-1.0,1.0,0.2,0
2,138074.0,"""\n\n\n\nI'm not sure if it's properly called ...",2002,1,article,random,test,10.0,0.0,1.0,0.5,0
3,200664.0,\n\n\n \nThanks on the info on how to move a p...,2002,1,user,random,test,10.0,0.0,1.0,0.4,0
4,213105.0,"""\n\n: I should do that too, I agree, but I've...",2002,1,user,random,test,10.0,0.0,1.0,0.3,0


Let's grab our best Logistic Regression model from notebook 2B-jkk-logistic-regression.

In [5]:
rs = load('../results/rs_cv_lr.joblib')
pipe = rs.best_estimator_
pipe.steps

[('vect',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=5,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='[a-z]+', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('select',
  SelectPercentile(percentile=10,
           score_func=<function chi2 at 0x000001F52A1C72F0>)),
 ('clf',
  LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]

Before we go any further, let's go ahead and generate predictions on the test set so we have some examples for later.

In [6]:
df_test["y_prob"] = pipe.predict_proba(df_test["comment"])[:,1]
df_test["y_pred"] = (df_test["y_prob"] > 0.5).astype(int)
df_test.head()

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y,y_prob,y_pred
0,8953.0,Elected or Electoral? JHK,2002,0,article,random,test,10.0,0.0,1.0,0.1,0,0.128771,0
1,28959.0,Please relate the ozone hole to increases in c...,2002,1,article,random,test,10.0,-1.0,1.0,0.2,0,0.555851,1
2,138074.0,"""\n\n\n\nI'm not sure if it's properly called ...",2002,1,article,random,test,10.0,0.0,1.0,0.5,0,0.029366,0
3,200664.0,\n\n\n \nThanks on the info on how to move a p...,2002,1,user,random,test,10.0,0.0,1.0,0.4,0,0.014342,0
4,213105.0,"""\n\n: I should do that too, I agree, but I've...",2002,1,user,random,test,10.0,0.0,1.0,0.3,0,0.024154,0


Our best result from the random search has a select step, which means that we will need to apply the mask obtained in the select step to the output of the vectorizer. Let's start by grabbing copies of the named steps in the pipeline, to make things easier to work with.

In [7]:
vect = pipe.named_steps['vect']
select = pipe.named_steps['select']
clf = pipe.named_steps['clf']

In [8]:
len(vect.vocabulary_)

30713

In [9]:
len(clf.coef_[0])

3072

We know that the select step only keeps 10% of the input features. We can extract the mask from the SelectPercentile object, but it is a hidden variable (note the leading underscore).

In [10]:
vect_mask = select._get_support_mask()
vect_mask

array([False, False, False, ..., False, False, False])

In [11]:
vect_mask.mean()

0.10002279165174356

From here, let's dump the vocabulary and coefficient for each term from the model so that we can examine **global** behavior.

In [12]:
vocab = vect.get_feature_names()
vocab = np.array(vocab)[vect_mask]
vocab[:5]

array(['abd', 'able', 'abomination', 'about', 'above'], dtype='<U23')

In [13]:
df_global = pd.DataFrame(
    {
        "token":vocab,
        "coef":clf.coef_[0]
    }
)
df_global.head()

,token,coef
0,abd,1.855428
1,able,-1.043615
2,abomination,5.311582
3,about,-1.280364
4,above,-0.534324


We can now sort by coef to examine the most important factors to the non-toxic and toxic classes. High coefficients are associated with the toxic class. Low (i.e., negative) coefficients are associated with the non-toxic class. Let's start by looking at the non-toxic class.

In [14]:
df_global = df_global.sort_values("coef", ascending=True)
df_global.head(10)

,token,coef
2692,thank,-9.985196
2693,thanks,-8.721378
2367,setting,-5.214016
1649,merged,-4.875996
2604,success,-4.840157
1810,noteworthy,-4.287727
2004,plural,-4.203437
1185,health,-4.136181
1213,historic,-4.118104
938,feedback,-4.087447


Nice, what about the other end?

In [15]:
df_global.tail(10)

,token,coef
1700,moron,13.833806
332,bullshit,14.714231
1280,idiot,15.148265
243,bitch,15.525072
557,crap,15.639011
139,ass,15.884015
2591,stupid,17.277218
2389,shit,19.169719
1027,fucking,21.629195
1018,fuck,27.375162


Well, this aligns with expectations. On the other hand, these are really **global** weights, and while they are useful for understanding general model behavior, what if we want to understand a **specific** prediction? In that case, we need to compute the activation, or the product of the coefficient with the weight from the bag of words representation. Let's write a helper function to do just that.

In [16]:
def get_activations(text, vect=vect, select=select, clf=clf):
    """Returns a list of tokens, their coefficients, and output activations for a given input."""
    x = vect.transform([text])
    x_sub = select.transform(x)
    x_out = np.multiply(x_sub.toarray()[0], clf.coef_[0])
    vect_mask = select._get_support_mask()
    vocab = vect.get_feature_names()
    vocab = np.array(vocab)[vect_mask]
    df_local = pd.DataFrame(
        {
            "token":vocab,
            "weight":x_sub.toarray()[0],
            "coef":clf.coef_[0]
        }
    )
    df_local = df_local[df_local["weight"] != 0]
    df_local["activation"] = df_local["weight"]*df_local["coef"]
    return df_local

How does it work? Let's grab an example from the test set where we successfully predicted the correct answer.

In [17]:
idx = df_test["y"] == df_test["y_pred"]
df_test[idx].sample(10, random_state=seed)

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y,y_prob,y_pred
3167,52759277.0,Perhaps the overwhelming evidence points to Ko...,2006,1,article,random,test,10.0,0.0,1.0,0.7,0,0.015011,0
27401,566125445.0,"""\n\n== Pakistan ==\n\nPlease do not add to a...",2013,1,user,random,test,10.0,-1.0,1.0,0.1,0,0.100116,0
19996,360915404.0,Go fuck yourself you stupid bitch go do someth...,2010,0,user,blocked,test,10.0,-2.0,-1.0,-1.6,1,0.999996,1
1295,27406343.0,you need to fuck off and stop hrassing people\n,2005,0,user,blocked,test,10.0,-2.0,-1.0,-1.3,1,1.000000,1
18505,325668254.0,\n:::: The problem with any use of the word 'i...,2009,1,article,blocked,test,10.0,0.0,1.0,0.4,0,0.060003,0
5706,86778622.0,"A tag has been placed on Snake Net, requesting...",2006,1,user,random,test,10.0,0.0,1.0,0.7,0,0.000759,0
2167,39845384.0,"\n\n== Alumni lists ==\n\nHi coblin, check out...",2006,1,user,random,test,10.0,0.0,2.0,1.0,0,0.021811,0
17302,300709731.0,\n\n== More efficient answer ==\n\nWhat you wr...,2009,1,user,random,test,10.0,-1.0,1.0,0.1,0,0.024006,0
29846,637441859.0,""" \n\n Thanks for the review and comments, my ...",2014,1,user,random,test,10.0,0.0,1.0,0.8,0,0.003742,0
25976,518999012.0,"\n\n:Whatever, man. I wouldn't call that edit ...",2012,1,user,blocked,test,10.0,-1.0,1.0,-0.1,1,0.769937,1


In [18]:
sample_text = df_test.loc[25976, "comment"]
print(sample_text)



:Whatever, man. I wouldn't call that edit warring, and yeah, I had no idea you couldn't point out that someone is lying when they are. It's a shame because I give a lot to this site. I make improvements where I see they're needed. This block and unnecessary and quite silly to be honest. But whatever. You're the man with the power.   


In [19]:
df_local = get_activations(sample_text)
df_local

,token,weight,coef,activation
76,and,0.137574,0.467370,0.064298
109,are,0.064288,2.153097,0.138418
207,be,0.059559,0.746915,0.044485
258,block,0.125874,1.149485,0.144690
342,but,0.068425,-2.917966,-0.199660
350,call,0.142085,0.183539,0.026078
808,edit,0.095172,-0.243452,-0.023170
1078,give,0.126470,0.276692,0.034993
1141,had,0.104769,0.346565,0.036309
1274,i,0.221984,-0.506888,-0.112521


Finally, let's use this information to augment the original text and highlight the keywords that are driving the decision.

In [20]:
target = "lying"
activation = 0.547197
sample_text_out = re.sub(f"\\b{target}\\b", f"<mark>{target}</mark>", sample_text)
HTML(sample_text_out)

We can expand this methodology to create a heat-map showing where the components of the message associated with each class are located.

In [21]:
def highlight_text(text, df_local):
    # Create a scaling factor for this local explanation
    alpha = 0.5/np.max(np.abs(df_local['activation']))
#     text_out = text
    for token, activation in df_local[["token", "activation"]].values:
        # Start by locating all instances of the token (ignoring case)
        targets = re.findall(f"\\b{token}\\b", text, flags=re.IGNORECASE)
        targets = set(targets)
        # Loop over each unique case and sub with a mark tag that is color-coded to indicate class contribution
        for target in targets:
            if activation > 0:
                text = re.sub(
                    f"\\b{target}\\b",
                    f"""<mark style="background-color:rgba(255,0,0,{alpha*activation});">{target}</mark>""",
                    text
                )
            else:
                text = re.sub(
                    f"\\b{target}\\b",
                    f"""<mark style="background-color:rgba(0,0,255,{-1*alpha*activation});">{target}</mark>""",
                    text
                )
    return text

text_out = highlight_text(sample_text, df_local)
HTML("<pre>" + text_out + "</pre>")

It works! What about a more nuanced example?

In [22]:
idx = (df_test["y_prob"] > 0.49) & (df_test["y_prob"] < 0.51) & (df_test["comment"].str.len() > 500)
df_test[idx].sample(10, random_state=seed)

,rev_id,comment,year,logged_in,ns,sample,split,num,min,max,avg,y,y_prob,y_pred
29743,634282094.0,"""\n\n== The American Agenda to Homosexualize t...",2014,1,article,blocked,test,10.0,-2.0,1.0,-0.4,1,0.509101,1
6812,105173339.0,\n\nI find your accusations of vandalism rudel...,2007,0,user,blocked,test,10.0,-1.0,1.0,0.1,0,0.506900,1
7819,122922847.0,April 2007 (UTC)\n\n::I got banned for being ...,2007,0,article,blocked,test,10.0,-1.0,1.0,0.1,0,0.492678,0
28073,588021858.0,"\n\n::Ha, what you said was a lie, when you ma...",2013,0,user,blocked,test,10.0,-1.0,1.0,-0.4,1,0.500344,1
26208,527241171.0,""", 9 December 2012 (UTC)\n\n::I find it far fr...",2012,0,article,blocked,test,10.0,-2.0,1.0,0.1,0,0.493297,0
10653,176630870.0,\n\n== Congratulations! ==\n\nI have awarded y...,2007,1,user,blocked,test,10.0,0.0,2.0,0.6,0,0.495029,0
4559,69491148.0,"""You lying SOB==\n""""I don't focus on single ar...",2006,0,user,blocked,test,10.0,-2.0,1.0,-0.6,1,0.499364,0
10193,168373112.0,"""\n\nI think the entire article should be tagg...",2007,1,article,random,test,10.0,-1.0,1.0,0.2,0,0.490501,0
21004,384129495.0,"""\n\n:::If Either way chooses to disregard my ...",2010,1,user,blocked,test,10.0,-1.0,1.0,0.3,0,0.490374,0
17743,308970708.0,"""\n\n==I need """"temperance""""?==\n\nYou don't k...",2009,1,user,blocked,test,10.0,-1.0,1.0,0.0,0,0.499381,0


In [23]:
sample_text = df_test.loc[6812, "comment"]
df_local = get_activations(sample_text)
text_out = highlight_text(sample_text, df_local)
HTML("<pre>" + text_out + "</pre>")

This is a powerful method of building trust among end users, and is useful for model training. However, we should note that this method starts to break down when you have features that span the same token multiple times (i.e., n-grams!). This heat-map approach might require layering at the individual character level, but that is left as an exercise to the reader.